[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anawatbk/hospital-capacity-forecasting/blob/main/final_dev.ipynb)

# 🛏️📈 California COVID-19 Hospital Bed Capacity Forecasting 

Author: Anawat Putwanphen
----------

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-4">Preprocessing</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Fit-scikit-learn-model-5">Feature Engineering</a></span></li><li><span><a 
href="#Fitting-and-Optimizing-Machine-Learning-Models-&-Hyperparameters" data-toc-modified-id="Evaluation-Metric-6">Fitting and Optimizing Machine Learning Models & Hyperparameters</a></span></li>
<li><span><a href="#Models-Evaluation" data-toc-modified-id="Evaluation-Metric-6">Models Evaluation</a></span></li><li><span><a 
href="#Conclusion" data-toc-modified-id="Evaluation-Metric-6">Conclusion</a></span></li></ul></div>

Project Goal
----
Forecast California COVID-19 Hospital bed capacity 1-month ahead (weekly average).



Dataset
-----

COVID-19 Reported (weekly average) Patient Impact and Hospital Capacity by Facility by Department of Health and Human Services   
https://healthdata.gov/dataset/covid-19-reported-patient-impact-and-hospital-capacity-facility
<br>
<br>
**Dimension:** 10,578 observations x 93 columns

In [148]:
import pandas as pd
import numpy as np

from sklearn.compose            import *
from sklearn.preprocessing      import *
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.impute import *
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline 

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Lasso, SGDRegressor, BayesianRidge
from sklearn.impute import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor

# Load Data
---------

In [28]:
data_source_path = '../data/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20210228.csv'
df = pd.read_csv(data_source_path, parse_dates = ['collection_week'])\
       .query('state == "CA"').sort_values(by=['hospital_pk', 'collection_week'])

# Preprocessing
--------------
Usually, Real world data is not in an ideal format for building machine learning models.  
In this section, I cleaned up some data quality issues from source data.  
For instance,
1. Some hospitals have reported more covid cases than combined total cases which consis of non-covid + covid cases by the source's definition
2. Select only hospitals who consistently have reported covid cases and bed capacity for the last 4 months (2020-11-06 until 2021-02-19)

Building 4-step ahead forecasting models require us to transform our original data into appropriate format for time series modelling.  
and clean up some data quality issues from source data.
This steps including
1. covert single target (time series) to 4-step-targets, for example, y | y+1 | y+2 | y+3
2. filter out rows for last 3 rows for each hospital since it wasn't possible to get y+1 to y+3 of the latest date

In [29]:
def pre_pipeline_preprocessing(df):
    '''
    All Preprocessing process which are not supported by Pipeline
    1. drop rows of inconsistent covid cases report,
       For example, covid cases > combined cases (which consist of non-covid + covid cases)
    2. select rows of hospital who report covid cases in the last 4 months
       (2020-11-06 until 2021-02-19) 
    '''
    # 1 Drop inconsistent covid cases report hospital (inpatient < covid patient)
    # column_sum/column_coverage(days) = weekly average
    inconsistent_hospital_pk = df[(
        (df['inpatient_beds_used_7_day_sum'] / df['inpatient_beds_used_7_day_coverage']) <      
        (df['total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum']
            / df['total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_coverage'])
        )]['hospital_pk'].unique() 
    df = df[~np.isin(df['hospital_pk'], inconsistent_hospital_pk)] # drop inconsistent report hospital

    # 2 keep only the hospitals who reported capacity every week for the last 4 month (2020-11-06 until 2021-02-19)
    max_week_count = df.loc[df['collection_week'] > '2020-11-01']\
                       .groupby('hospital_pk').count().max()['collection_week']
    hospital_pk_array = df.loc[df['collection_week'] > '2020-11-01']\
                          .groupby('hospital_pk').count().index.values
    complete_hostital_mask = (df.loc[df['collection_week'] > '2020-11-01']\
                                .groupby('hospital_pk').count()['collection_week'] == max_week_count)
    hospital_pk_array = hospital_pk_array[complete_hostital_mask]
    df = df[np.isin(df['hospital_pk'], hospital_pk_array)].copy()
    return df
    
def pre_pipeline_generate_multi_step_y(df):
    '''
    Generate  1 month-ahead (4 step-ahead) output target.
    y | y+1 | y+2 | y+3
    '''
    # Transform target y into 4-step-ahead
    y = df[['hospital_pk', 'inpatient_beds_used_7_day_sum', 
            'inpatient_beds_used_7_day_coverage']].copy()
    # Weekly average bed used = bed used sum / bed used average
    y['inpatient_bed_used'] = (y['inpatient_beds_used_7_day_sum'] / 
                               y['inpatient_beds_used_7_day_coverage'])
    # Clean up target y
    # By the source's definition -999999 values must be filled with 0
    # By the source's definition Nan values must be filled with 0 
    
    y['inpatient_bed_used'] = y['inpatient_bed_used'].fillna(0)
    y.loc[y['inpatient_bed_used'] < 0, 'inpatient_bed_used'] = 0  # fill -999999 values (or -114971 if average) with 0
    
    # Create leading columns --> yt+1, y+2, y+3
    # The data must be groupby hospital_pk otherwise, we will get bed_used number from other hospital
    step_ahead = 3
    col = 'inpatient_bed_used'
    for step in range(1, step_ahead+1):
        y = y.assign(**{f'{col}+{step}': y.groupby('hospital_pk').shift(-step)[col]}) 

    # Drop the last 3 weeks of each hospital rows (X and y)
    y = y.drop(['inpatient_beds_used_7_day_sum', 
                'inpatient_beds_used_7_day_coverage', 'hospital_pk'], 
               axis=1) # drop irrelavant columns from y
    no_target_week_mask = y.isnull().sum(axis=1) > 0 # boolean array of the last 3 weeks
    df = df[~no_target_week_mask].copy() 
    y = y.dropna()
    y = y[['inpatient_bed_used', 'inpatient_bed_used+1', 
           'inpatient_bed_used+2', 'inpatient_bed_used+3']]
    df['inpatient_bed_used'] = y['inpatient_bed_used'] # keep y in X to use for lagging features
    return df.reset_index(drop=True), y.reset_index(drop=True)

In [30]:
df = pre_pipeline_preprocessing(df)
X_original, y_original = pre_pipeline_generate_multi_step_y(df)

# Feature Engineering
----------------
Features for machine learning model are extracted in this step.
The process including
<br>
**1. Create custom calculated columns**
<br>
Some information in Raw data can not be used directly for features. Some calculation must be performed.
- Inpatient Bed Used (weekly average)
- Total Covid inpatient (weekly average)
- Inpatient Bed used (hospital average)

**2. Select only columns relevant for this project**
<br>
<br>
**3. Generate Lag features**
<br>
Example: x-4 | x-3 | x-2 | x-1
<br>
Create lag features for time series column including inpatient bed used, total covid inpatient.
<br>
<br>
**4. Groupby Imputer**
<br>
Impute missing values of each column by its hospital's median .

### Create custom calculated columns

In [31]:
class CreateCalculatedColumns(BaseEstimator, TransformerMixin):
    ''' 
    Create new calculated columns pipelines.
    
    Parameters
    ----------
    None
    
    '''
    def __init__(self):
        pass

    def transform(self, X, **transform_params):
        """ 
        
        Parameters
        ----------
        X : pandas DataFrame
            
        Returns
        ----------
        
        trans : pandas DataFrame
            DataFrame contain new calculated columns.     
        """
        self.before_shape = X.shape
        
        # Inpatient Bed Capacity (use to summarize capacity %)
        X['inpatient_bed_capacity'] = X['inpatient_beds_7_day_sum'] / X['inpatient_beds_7_day_coverage']
        # ICU Bed Capacity (use to summarize capacity %)
        X['icu_bed_capacity'] = X['total_icu_beds_7_day_sum'] / X['total_icu_beds_7_day_coverage']
        # Inpatient Bed Used
        X['inpatient_bed_used'] = X['inpatient_beds_used_7_day_sum'] / X['inpatient_beds_used_7_day_coverage']
        # Adult covid inpatient   
        X['adult_inpatients_confirmed_n_suspected_covid'] = (
            X['total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum'] 
             / X['total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_coverage'])
        # Pediatric covid inpatient   
        X['pediatric_inpatrients_confirmed_n_suspected_covid'] = (
            X['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum']
            / X['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_coverage'])
        # Total covid inpatient (Adult+Pediatric)
        X['total_inpatients_confirmed_n_suspected_covid'] = (
            X['adult_inpatients_confirmed_n_suspected_covid'] 
            + X['pediatric_inpatrients_confirmed_n_suspected_covid'])
        # From source's definition. Fill -999999 values by 0 values
        X.loc[X['total_inpatients_confirmed_n_suspected_covid'] < 0, 'total_inpatients_confirmed_n_suspected_covid'] = 0
        X.loc[X['inpatient_bed_used'] < 0, 'inpatient_bed_used'] = 0
        # Average Inpatient Bed Used of each hospital
        avg_inpatient_bed_used = X.groupby('hospital_pk').mean()[['inpatient_bed_used']]\
                                  .rename(columns={'inpatient_bed_used':'avg_inpatient_bed_used'})
        
        X = X.join(avg_inpatient_bed_used, on='hospital_pk')
        trans = X.copy() 
        self.after_shape = trans.shape
        return trans

    def fit(self, X, y=None, **fit_params):
        """ Do nothing function
        
        Parameters
        ----------
        X : pandas DataFrame
        y : default None
                
        
        Returns
        ----------
        self  
        """
        return self

### Select only columns relevant for this project 

1. hospital primary key: 'hospital_pk'
2. start of the week date: 'collection_week'
3. Sub type of hospital: 'hospital_subtype'
4. Type of hospital (metro/micro): 'is_metro_micro'
5. Hospital's Inpatient Bed Capacity: 'inpatient_bed_capacity'
6. Hospital's Inpatient Bed usage: 'inpatient_bed_used'
7. Average Inpatient Bed usage: 'avg_inpatient_bed_used'
8. Hospital's COVID Inpatient (confirmed & suspected): 'total_inpatients_confirmed_n_suspected_covid'

In [32]:
# Specify relavant columns and non-features column
relevant_columns = [
    'hospital_pk', 'collection_week', 'hospital_subtype', 'is_metro_micro',
    'inpatient_bed_capacity', 'icu_bed_capacity', 
    'inpatient_bed_used', 'avg_inpatient_bed_used',
    'total_inpatients_confirmed_n_suspected_covid',
    ]

# non features - hospital key, collection date, full capacity data (icu & Inpatient)
non_feature_columns = ['hospital_pk', 'collection_week', 
                       'inpatient_bed_capacity', 'icu_bed_capacity']

In [33]:
class SelectColumnsTransfomer(BaseEstimator, TransformerMixin):
    """ 
    
    Select columns by column names from pandas dataframes in scikit-learn pipelines.
    
    Parameters
    ----------
    columns : list of column names 
    feature : Boolean
        if True drop specify non-feature columns instead
        else select specify relavant columns.
    
    """
    def __init__(self, columns=[], feature=False):
        self.columns = columns
        self.feature = feature
    def transform(self, X, **transform_params):
        """ 
        Selects columns of a DataFrame

        Returns
        ----------
        trans : pandas DataFrame
        pandas DataFrame contains selected columns of X      
        """
        if self.feature:
            X = X.drop(self.columns, axis=1)
            return X
        else: 
            X = X[self.columns].copy() 
            return X

    def fit(self, X, y=None, **fit_params):
        """ Do nothing function
        
        Parameters
        ----------
        X : pandas DataFrame
        y : default None
                
        
        Returns
        ----------
        self  
        """
        return self

### Generate Lag features 
Generate lag features for 
1. Hospital's Inpatient bed usage 
2. Hospital's Covid Inpatient cases.

In [34]:
time_dependant_columns = ['inpatient_bed_used', 'total_inpatients_confirmed_n_suspected_covid']

In [35]:
class GenerateLagValues(BaseEstimator, TransformerMixin):
    """ A DataFrame transformer that provides column selection
    
    Select columns by column names from pandas dataframes in scikit-learn pipelines.
    
    Parameters
    ----------
    columns : list of str, names of the dataframe columns to select
    lags : specify how much lag values to calculate
        Example lags = 4 create
        x-4 | x-3 | x-2 | x-1
    
    """
    def __init__(self, columns=[], lags=4):
        self.columns = columns
        self.lags = lags
    def transform(self, X, **transform_params):
        """ Selects columns of a DataFrame
        
        Parameters
        ----------
        X : pandas DataFrame
            
        Returns
        ----------
        
        trans : pandas DataFrame
            contains selected columns of X      
        """
        self.before_shape = X.shape
        
        for col in self.columns:
            for lag in range(1, self.lags+1):
                X = X.assign(**{f'{col}-{lag}': X.groupby('hospital_pk').shift(lag)[col]})
        
        return X.drop(self.columns, axis=1)

    def fit(self, X, y=None, **fit_params):
        """ Do nothing function
        
        Parameters
        ----------
        X : pandas DataFrame
        y : default None
                
        
        Returns
        ----------
        self  
        """
        return self

### Groupby Imputer
Impute missing value by the median value of the hospital

In [36]:
class GroupByImputer(BaseEstimator, TransformerMixin):
    '''
    using median of group to impute (hospital_pk)
    fill by 0 if all values in the group are Nan
    '''
    def __init__(self, group_column, targets=[]):
        self.group_column = group_column
        self.targets = targets
    
    def fit(self, X, y=None):
        
        impute_map = X.groupby(self.group_column)[self.targets].median().reset_index(drop=False)
        self.impute_map_ = impute_map
        
        return self 
    
    def transform(self, X, y=None):

        X = X.copy()
        
        for index, row in self.impute_map_.iterrows(): # loop through each hospital
            group_mask = row[self.group_column] == X[self.group_column]
            for col in self.targets:
                X.loc[group_mask, col] = X.loc[group_mask, col].fillna(row[col])
        X[self.targets] = X[self.targets].fillna(0)
        return X

# Time Series training/validation/testing Split
-----------------------------------


#### Splitting Strategy
<pre>
Training:   2020-07-31 to 2021-12-25 (5 months)  
Validation: 2021-01-01 to 2021-01-22 (1 month)  
Testing:    2021-01-29 to 2021-02-19 (1 month)
</pre>

In [265]:
def time_series_split(X, y):
    test_start_date =  X.collection_week.max()
    train_last_date  = test_start_date - pd.to_timedelta(4,unit='W')    
    test_idxs = X.loc[X.collection_week >= test_start_date].index
    train_idx = X.loc[X.collection_week <= train_last_date].index
    X_test, y_test = X.loc[test_idxs], y.loc[test_idxs]
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    return X_train, y_train, X_test, y_test

def time_series_cv(X, y):
    '''
    Create the custom Time Series cross validation object
    Training:   2020-07-31 to 2021-12-25 (5 months) 
    Validation: 2021-01-01 to 2021-01-22 (1 month)  
    '''
    X = X.reset_index(drop=True)
    valid_start_date =  X.collection_week.max()
    train_last_date  = valid_start_date - pd.to_timedelta(4,unit='W') 
    valid_idxs = X.loc[X.collection_week >= valid_start_date].index
    train_idx = X.loc[X.collection_week <= train_last_date].index
    return [tuple([list(train_idx), list(valid_idxs)])]

In [38]:
X_train, y_train, X_test, y_test = time_series_split(X_original, y_original)

# Fitting and Optimizing Machine Learning Models & Hyperparameters
-----------

## 1 Randomize Search for the best ML model & hyperparameter
### Model Candidates

**1. Lasso (Multi-output)**
<br>
* Hyperparameters candidates  
 *  **alpha**: Different values of regularized coefficient(alpha) were choosen to explore the effect of regularization.

**2. SGDRegressor (Multi-output)**
* Hyperparameters candidates
 * **loss**: 2 types of loss functions (huber vs least sqaure) were choosen to explore the performance.
 * **penalty**: 2 types of regularization were choose to explore its effect (l2 & ElasticNet).
 * **alpha**: Different values of regularized coefficient(alpha) were choosen to explore the effect of regularization.
 
**3. RandomForestRegressor (Multi-output)**
* Hyperparameters candidates
 * **max_features**: 'sqrt', 'log2', 0.333 were choose to explore the effect of reducing trees correlation (reducing variance) by limiting max features
 * **n_estimators**: Different number of estimators were choosen to explore the performance.
 * **max_depth**: Different number of max depth were choosen to explore overfitting effect.
 * **min_samples_leaf**: Different number of min sampels leaf were choosen to explore overfitting effect.

**4. GradientBoostingRegressor  (Multi-output)**
* Hyperparameters candidates
 * **max_features**: 'sqrt', 'log2', 0.333 were choose to explore the effect of reducing trees correlation (reducing variance) by limiting max features.
 * **n_estimators**: Different number of estimators were choosen to explore the performance.
 * **max_depth**: Different number of max depth were choosen to explore overfitting effect.
 * **min_samples_leaf**: Different number of min sampels leaf were choosen to explore overfitting effect.
 * **loss**: 2 types of loss functions (huber vs least sqaure) were choosen to explore the performance.
 * **subsample**: Different subsample ratio were choose to explore if it could help to reduce the variance.

In [206]:
class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass

# Continuous value columns pipeline
con_pipe = Pipeline([('scaler', StandardScaler())]) 
feature_lag_cols = []
lags = 5
for col in time_dependant_columns:
    for lag in range(1, lags+1):
        feature_lag_cols.append(f'{col}-{lag}')

feature_con_cols = feature_lag_cols + ['avg_inpatient_bed_used']

# Categorical value columns pipeline
feature_cat_cols = ['hospital_subtype', 'is_metro_micro']
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy="most_frequent", add_indicator=True)),
                     ('ohe', OneHotEncoder())])

# Combine features pipeline
to_feature = ColumnTransformer([('continuous',  con_pipe, feature_con_cols),
                                ('categorical', cat_pipe, feature_cat_cols)])

# Final pipeline
pipeline = Pipeline([('calculateColumns', CreateCalculatedColumns()),
                             ('selectColumns_1', SelectColumnsTransfomer(relevant_columns)),
                             ('createTimelag', GenerateLagValues(time_dependant_columns, lags=lags)),
                             ('custom_imputer', GroupByImputer('hospital_pk', targets=feature_con_cols)),
                             ('selectColumns_2', SelectColumnsTransfomer(non_feature_columns, feature=True)),
                             ('finalProcessing', to_feature),
                             ('model', DummyEstimator)])

In [207]:
# Search Best Models
cv_time_series = time_series_cv(X_train, y_train)
hyperparameters = [{'model': [Lasso(max_iter=3000)],
                    'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1 ,10]},
                   
                   {'model': [SGDRegressor(max_iter=3000, early_stopping=True)],
                    'model__loss': ['squared_loss', 'huber'],
                    'model__penalty': ['l2', 'elasticnet'],
                    'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1 ,10]},
                   
                   {'model': [RandomForestRegressor()],
                    'model__max_features': ['sqrt', 'log2', 0.333],
                    'model__n_estimators': np.arange(25, 201, 25),
                    'model__max_depth': np.arange(10,31,5),
                    'model__min_samples_leaf': [1, 3, 5, 10, 25, 100]},
    
                    {'model': [GradientBoostingRegressor()],
                     'model__max_features': ['sqrt', 'log2', 0.333],
                     'model__loss': ['ls', 'huber'],
                     'model__n_estimators': np.arange(25,201,25),
                     'model__max_depth': np.arange(10,31,5),
                     'model__subsample': [0.8,0.9,1.0],
                     'model__min_samples_leaf': np.arange(1,16,3)}]

regr_rand_cv = RandomizedSearchCV(estimator=pipeline, 
                              param_distributions=hyperparameters, 
                              n_iter=100, 
                              cv=cv_time_series, 
                              scoring='neg_mean_absolute_error',
                              n_jobs=-1,
                              verbose=True)

In [208]:
regr_rand_cv.fit(X_train, y_train)

Fitting 1 folds for each of 100 candidates, totalling 100 fits


/Users/bank/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [         nan -21.22827525 -20.77078304          nan          nan
 -19.5893402           nan          nan          nan -22.10603561
 -19.34124895 -20.50383201          nan          nan          nan
          nan -20.21889359          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan -19.70377385          nan          nan
 -20.36520721          nan          nan          nan -21.03207205
 -19.92231546          nan          nan -20.62294273 -19.95779161
          nan          nan          nan          nan          nan
          nan          nan          nan          nan -20.40146533
          nan          nan          nan          nan      

RandomizedSearchCV(cv=[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                         16, 17, 18, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, ...],
                        [22, 45, 68, 91, 114, 137, 160, 183, 206, 229, 252, 275,
                         298, 321, 344, 367, 390, 413, 436, 459, 482, 505, 528,
                         551, 574, 597, 620, 643, 666, 689, ...])],
                   estimator=Pipeline(steps=[('calculateColumns',
                                              CreateCalculatedColumns()),
                                             ('selectColumns_1',
                                              SelectColumnsTransfomer(...
                                         'model__n_estimators': array([ 25,  50,  75, 100, 125, 150, 175, 200])},
                                        {'model': [GradientBoostingRegressor()],
                                         'model__loss': ['ls', 'huber'],
                                         'model__max_depth

In [209]:
regr_rand_cv.best_params_

{'model__n_estimators': 100,
 'model__min_samples_leaf': 5,
 'model__max_features': 0.333,
 'model__max_depth': 25,
 'model': RandomForestRegressor(max_depth=25, max_features=0.333, min_samples_leaf=5)}

#### Fit Final pipeline using best Hyperparameters

In [212]:
pipeline_final = Pipeline([('calculateColumns', CreateCalculatedColumns()),
                             ('selectColumns_1', SelectColumnsTransfomer(relevant_columns)),
                             ('createTimelag', GenerateLagValues(time_dependant_columns, lags=lags)),
                             ('custom_imputer', GroupByImputer('hospital_pk', targets=feature_con_cols)),
                             ('selectColumns_2', SelectColumnsTransfomer(non_feature_columns, feature=True)),
                             ('finalProcessing', to_feature),
                             ('model', RandomForestRegressor(max_depth=25, max_features=0.333, 
                                                             min_samples_leaf=5, n_estimators=100))])
pipeline_final.fit(X_train, y_train)

Pipeline(steps=[('calculateColumns', CreateCalculatedColumns()),
                ('selectColumns_1',
                 SelectColumnsTransfomer(columns=['hospital_pk',
                                                  'collection_week',
                                                  'hospital_subtype',
                                                  'is_metro_micro',
                                                  'inpatient_bed_capacity',
                                                  'icu_bed_capacity',
                                                  'inpatient_bed_used',
                                                  'avg_inpatient_bed_used',
                                                  'total_inpatients_confirmed_n_suspected_covid'])),
                ('createTimelag',
                 GenerateLa...
                                                   'total_inpatients_confirmed_n_suspected_covid-4',
                                                   'total_inpatien

## 2 Stacking models with a Metalearner

Stack 3 different models including
1. Lasso
2. Bayesian
3. RandomForest

#### 2.1 Search for the best of each model

Lasso

In [147]:
hyperparameters_lasso = [{'model': [Lasso(max_iter=3000)],
                    'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1 ,10]}]

best_lasso_cv = RandomizedSearchCV(estimator=pipeline, 
                              param_distributions=hyperparameters_lasso, 
                              n_iter=6, 
                              cv=cv_time_series, 
                              scoring='neg_mean_absolute_error',
                              n_jobs=-1,
                              verbose=True)
best_lasso_cv.fit(X_train, y_train)
best_lasso_cv.best_params_

Fitting 1 folds for each of 6 candidates, totalling 6 fits


{'model__alpha': 0.0001, 'model': Lasso(alpha=0.0001, max_iter=3000)}

**SGDRegressor**

In [161]:
hyperparameters_sgd = [{'model': [MultiOutputRegressor(SGDRegressor(early_stopping=True))],
                    'model__estimator__loss': ['huber'],
                    'model__estimator__penalty': ['l2', 'elasticnet'],
                    'model__estimator__alpha': [0.0001, 0.001, 0.01, 0.1, 1 ,10]}]

best_sgd_cv = RandomizedSearchCV(estimator=pipeline, 
                              param_distributions=hyperparameters_sgd, 
                              n_iter=12, 
                              cv=cv_time_series, 
                              scoring='neg_mean_absolute_error',
                              n_jobs=-1,
                              verbose=True)
best_sgd_cv.fit(X_train, y_train)
best_sgd_cv.best_params_

Fitting 1 folds for each of 12 candidates, totalling 12 fits


{'model__estimator__penalty': 'l2',
 'model__estimator__loss': 'huber',
 'model__estimator__alpha': 0.0001,
 'model': MultiOutputRegressor(estimator=SGDRegressor(early_stopping=True, loss='huber'))}

**RandomForest**

In [162]:
hyperparameters_rf = [{'model': [RandomForestRegressor()],
                    'model__max_features': ['sqrt', 'log2', 0.333],
                    'model__n_estimators': np.arange(25, 201, 25),
                    'model__max_depth': np.arange(10,31,5),
                    'model__min_samples_leaf': [1, 3, 5, 10, 25, 100]}]

best_sgd_rf = RandomizedSearchCV(estimator=pipeline, 
                              param_distributions=hyperparameters_rf, 
                              n_iter=12, 
                              cv=cv_time_series, 
                              scoring='neg_mean_absolute_error',
                              n_jobs=-1,
                              verbose=True)
best_sgd_rf.fit(X_train, y_train)
best_sgd_rf.best_params_

Fitting 1 folds for each of 12 candidates, totalling 12 fits


{'model__n_estimators': 75,
 'model__min_samples_leaf': 3,
 'model__max_features': 0.333,
 'model__max_depth': 15,
 'model': RandomForestRegressor(max_depth=15, max_features=0.333, min_samples_leaf=3,
                       n_estimators=75)}

#### Search for best hyperparameters

In [254]:
estimators = [('ridge', Lasso(max_iter=3000,alpha=0.0001)),
              ('SGDRegressor', SGDRegressor(max_iter=3000, 
                                            penalty='l2', 
                                            loss='huber', 
                                            alpha=0.0001)),
              ('rf',   RandomForestRegressor(max_depth=25, max_features=0.333, 
                                             min_samples_leaf=5, n_estimators=100))]

final_estimator = GradientBoostingRegressor()

stacking_regressor = MultiOutputRegressor(
    StackingRegressor(estimators=estimators,
                      final_estimator=final_estimator))

hyperparameters = [{'model': [stacking_regressor],
                    'model__estimator__final_estimator__n_estimators': [50,100,150,200],
                    'model__estimator__final_estimator__max_features': [0.33, 'auto', 'sqrt', 'log2'],
                    'model__estimator__final_estimator__max_depth': np.arange(10,31,5),
                    'model__estimator__final_estimator__min_samples_leaf': [1, 3, 5, 10, 25]}]

best_final_stacking = RandomizedSearchCV(estimator=pipeline, 
                                      param_distributions=hyperparameters, 
                                      n_iter=20, 
                                      cv=cv_time_series, 
                                      scoring='neg_mean_absolute_error',
                                      n_jobs=-1,
                                      verbose=True)
best_final_stacking.fit(X_train, y_train)

Fitting 1 folds for each of 20 candidates, totalling 20 fits


/Users/bank/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
  warnings.warn(


RandomizedSearchCV(cv=[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                         16, 17, 18, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, ...],
                        [22, 45, 68, 91, 114, 137, 160, 183, 206, 229, 252, 275,
                         298, 321, 344, 367, 390, 413, 436, 459, 482, 505, 528,
                         551, 574, 597, 620, 643, 666, 689, ...])],
                   estimator=Pipeline(steps=[('calculateColumns',
                                              CreateCalculatedColumns()),
                                             ('selectColumns_1',
                                              SelectColumnsTransfomer(...
                                                                                                                                              n_estimators=50)))],
                                         'model__estimator__final_estimator__max_depth': array([10, 15, 20, 25, 30]),
                                         'mod

In [244]:
best_final_stacking.best_params_

{'model__estimator__final_estimator__n_estimators': 200,
 'model__estimator__final_estimator__min_samples_leaf': 5,
 'model__estimator__final_estimator__max_depth': 20,
 'model__estimator__final_estimator__loss': 'ls',
 'model': MultiOutputRegressor(estimator=StackingRegressor(estimators=[('ridge',
                                                               Lasso(alpha=0.0001,
                                                                     max_iter=3000)),
                                                              ('SGDRegressor',
                                                               SGDRegressor(loss='huber',
                                                                            max_iter=3000)),
                                                              ('rf',
                                                               RandomForestRegressor(max_depth=25,
                                                                                     max_features=0.3

**Fit Final pipeline using best parameters**

In [259]:
final_estimator = GradientBoostingRegressor(max_features=0.333,
                                            min_samples_leaf=25,
                                            n_estimators=150)


stacking_regressor = MultiOutputRegressor(
    StackingRegressor(estimators=estimators,
                      final_estimator=final_estimator)
    )

pipeline_stacking = Pipeline([('calculateColumns', CreateCalculatedColumns()),
                             ('selectColumns_1', SelectColumnsTransfomer(relevant_columns)),
                             ('createTimelag', GenerateLagValues(time_dependant_columns, lags=lags)),
                             ('custom_imputer', GroupByImputer('hospital_pk', targets=feature_con_cols)),
                             ('selectColumns_2', SelectColumnsTransfomer(non_feature_columns, feature=True)),
                             ('finalProcessing', to_feature),
                             ('model', stacking_regressor)])
pipeline_stacking.fit(X_train, y_train)

Pipeline(steps=[('calculateColumns', CreateCalculatedColumns()),
                ('selectColumns_1',
                 SelectColumnsTransfomer(columns=['hospital_pk',
                                                  'collection_week',
                                                  'hospital_subtype',
                                                  'is_metro_micro',
                                                  'inpatient_bed_capacity',
                                                  'icu_bed_capacity',
                                                  'inpatient_bed_used',
                                                  'avg_inpatient_bed_used',
                                                  'total_inpatients_confirmed_n_suspected_covid'])),
                ('createTimelag',
                 GenerateLa...
                ('model',
                 MultiOutputRegressor(estimator=StackingRegressor(estimators=[('ridge',
                                                     

In [260]:
# Fix Multioutput+Stacking Bug in Sci-kit Learn Ref: https://github.com/scikit-learn/scikit-learn/issues/16549
pipeline_stacking.steps[-1][1].estimator.estimators_ = pipeline_stacking.steps[-1][1].estimator.estimators
pipeline_stacking.steps[-1][1].estimator.final_estimator_ = pipeline_stacking.steps[-1][1].estimator.final_estimator

# Models Evaluation
----

**Evaluation Metric**
- Uniform average of **mean absolote error (MAE)** was chosen as evaluation metric as mean absolote error unit is easily interpret by layperson. The error unit is the number of bed usage that we wrongly predicted.

In [263]:
# Final best Model
y_pred = pipeline_final.predict(X_test)
mae_y_pred = mean_absolute_error(y_test.values, y_pred, multioutput='uniform_average')
print('MAE Best ML model(RF):', mae_y_pred)

MAE Best ML model(RF): 12.262304999642728


In [264]:
# Stacking Model
y_pred_stacking = pipeline_stacking.predict(X_test)
mae_y_pred_stacking = mean_absolute_error(y_test.values, y_pred_stacking, multioutput='uniform_average')
print('MAE Stacking Ensemble model:', mae_y_pred_stacking)

MAE Stacking Ensemble model: 12.4845304135933


In [262]:
# Naive Model
baseline = np.hstack([X_original[X_original.collection_week == '2021-01-22'][['inpatient_bed_used']].values]*4)
mae_baseline = mean_absolute_error(y_test.values, baseline, multioutput='uniform_average')
print('MAE Naive baseline (Forecast by latest week):', mae_baseline)

MAE Naive baseline (Forecast by latest week): 15.431180643348785


### MAE Comparison



**1. Best Model (Random Forest): 12.26**  
2. Stacking Ensemble: 12.48  
3. Naive Model: 15.43  

# Conclusion
------------

Different types of Machine learning model were choosen for comparison during randomized hyperparameters search.
For instance, 
* Regularlized Linear Regression (l2, l1, ElasticNet)
* Bagging Ensemble Model (RandomForest)
* Boosting Ensemble Model (GradientBoosting)  

<br>
After comparing all models using RandomizedSearchCV, RandomForest was selected as the final model for forecasting.
<br>
It achieved the lowest MAE (MAE=12.26), with comparison to the Naive model, it has 20% lower MAE.
<br>
In the latter section, the Stacking Ensemble Model from linear models & RandomForest was also tested.
<br>
The performance of the stacking ensemble model was very close to RandomForest model with MAE=12.48. 
<br>
The result from the experiment indicated that Ensemble Models perform well for forecasting hospital capacity


